In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3
import datetime
import re
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sqlite_api import SLOHouseDatabase
def classify_predict(df, X_vars, y_var,colName,SVC=True,kNN=True,Naive=True,Gaussian=True,DecTree=True):
    if type(X_vars) is not list:
        X = df[X_vars].reshape(-1,1)
    else:
        X = df[X_vars]
    y = df[y_var]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=26)
    length=len(df)
    
    print('Classification Accuracies')
    svc = SVC()
    svc.fit(X_train, y_train)
    initial=[[predict] for predict in svc.predict(X_test).iteritems()]
    
    print('Support Vector: ' + str(svc.score(X_test, y_test)))
    
    max_score = 0
    for i in range(1, 6):
        kNN = KNeighborsClassifier(n_neighbors=i)
        kNN.fit(X_train, y_train)
        cur_score = kNN.score(X_test, y_test)
        if cur_score > max_score:
            max_score = cur_score
            maxN = i
    kNN = KNeighborsClassifier(n_neighbors=maxN)
    kNN.fit(X_train, y_train)
    i=0
    for predict in kNN.fit(x_train).iteritems():
        initial[i].append(predict)
        i+=1
    print('kNN (N = '+str(maxN)+'): ' + str(max_score))
    
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    i=0
    for predict in dtc.fit(x_train).iteritems():
        initial[i].append(predict)
        i+=1
    print('Decision Tree: ' + str(dtc.score(X_test, y_test)))
    
    nbc = GaussianNB()
    nbc.fit(X_train, y_train)
    i=0
    for predict in nbc.fit(x_train).iteritems():
        initial[i].append(predict)
        i+=1
    print('Naive Bayes: '+ str(nbc.score(X_test, y_test)))
    
    gpc = GaussianProcessClassifier()
    gpc.fit(X_train, y_train)
    i=0
    for predict in gpc.fit(x_train).iteritems():
        initial[i].append(predict)
        i+=1
    print('Gaussian Process: '+ str(gpc.score(X_test, y_test)))
    prediction=[max(set(lst), key=lst.count) for lst in initial]
    